In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys, BinanceUSDMTestKeys, ApexTestKeys, MufexTestKeys
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from quantfreedom.exchanges.binance_exchange.binance_usdm import BinanceUSDM
from quantfreedom.enums import PositionModeType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.exchanges.apex_exchange.apex import Apex
from quantfreedom.indicators.tv_indicators import *
np.set_printoptions(formatter={"float_kind": "{:0.1f}".format})


%load_ext autoreload
%autoreload 2

In [3]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)
mufex_candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=200,
)

mufex_open = mufex_candles[:, 1]
mufex_high = mufex_candles[:, 2]
mufex_low = mufex_candles[:, 3]
mufex_close = mufex_candles[:, 4]


mufex_close_shift = np.roll(mufex_candles[:, 4], 1)
mufex_close_shift[0] = np.nan
mufex_close

array([36559.9, 36539.3, 36601.6, 36623.1, 36569.8, 36535.9, 36609.3,
       36653.6, 36698.2, 36759.1, 36753.7, 36791.2, 36720.5, 36669.4,
       36665.0, 36723.9, 36680.1, 36604.4, 36602.5, 36613.5, 36584.9,
       36524.7, 36570.0, 36524.2, 36513.5, 36539.3, 36555.6, 36567.1,
       36623.1, 36653.3, 36638.7, 36660.7, 36665.7, 36676.3, 36718.1,
       36696.0, 36701.8, 36643.8, 36697.0, 36660.4, 36728.4, 36738.2,
       36745.2, 36779.2, 36808.8, 36824.8, 36836.4, 36874.8, 36876.3,
       36832.6, 36888.7, 36923.3, 36858.9, 36804.3, 36801.5, 36760.0,
       36809.4, 36775.3, 36783.9, 36762.8, 36754.7, 36730.6, 36760.5,
       36727.1, 36708.5, 36750.4, 36737.0, 36778.1, 36791.2, 36704.1,
       36724.3, 36766.9, 36815.3, 36770.7, 36739.7, 36735.4, 36699.6,
       36728.3, 36710.4, 36698.3, 36703.9, 36744.5, 36734.4, 36750.1,
       36775.4, 36741.9, 36709.1, 36719.8, 36674.9, 36645.4, 36653.8,
       36652.2, 36639.2, 36652.4, 36677.9, 36650.8, 36633.8, 36701.0,
       36621.9, 3665

In [39]:
mufex_main.create_order(
    symbol="BTCUSDT",
    position_mode=PositionModeType.BuySide,
    buy_sell="buy",
    order_type="limit",
    price=35600,
    asset_size=0.005,
    time_in_force="GoodTillCancel",
)

'feb6f49b-db41-41ad-b15c-500a38747791'

In [40]:
dl_ex_candles(
    exchange="apex",
    symbol="BTCUSDC",
    timeframe="5m",
    candles_to_dl=200,
)

array([[1699572300000.0, 36577.5, 36661.0, 36577.0, 36649.0],
       [1699572600000.0, 36649.0, 36709.0, 36648.0, 36690.0],
       [1699572900000.0, 36690.0, 36750.0, 36690.0, 36740.5],
       [1699573200000.0, 36740.5, 36808.0, 36695.0, 36808.0],
       [1699573500000.0, 36808.0, 36808.0, 36739.5, 36805.5],
       [1699573800000.0, 36805.5, 36809.0, 36792.5, 36809.0],
       [1699574100000.0, 36809.0, 36809.0, 36736.0, 36754.5],
       [1699574400000.0, 36754.5, 36764.5, 36711.0, 36723.5],
       [1699574700000.0, 36723.5, 36771.5, 36698.0, 36722.5],
       [1699575000000.0, 36722.5, 36772.0, 36683.5, 36772.0],
       [1699575300000.0, 36772.0, 36809.0, 36728.0, 36735.0],
       [1699575600000.0, 36735.0, 36742.5, 36633.5, 36660.0],
       [1699575900000.0, 36660.0, 36693.0, 36636.0, 36652.0],
       [1699576200000.0, 36652.0, 36684.5, 36638.5, 36640.5],
       [1699576500000.0, 36640.5, 36659.0, 36607.0, 36629.0],
       [1699576800000.0, 36629.0, 36629.0, 36564.5, 36569.5],
       [

In [41]:
apex_test = Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    stark_key_private=ApexTestKeys.stark_key_private,
    stark_key_public=ApexTestKeys.stark_key_public,
    stark_key_y=ApexTestKeys.stark_key_y,
    use_test_net=True
)
apex_test.create_entry_market_order(
    symbol="BTC-USDC",
    buy_sell="sell",
    asset_size=0.005,
)

In [43]:
b_candles = dl_ex_candles(
    exchange="binance_usdm",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=200,
)
b_candles

In [44]:
candles_to_df(candles=b_candles)

,timestamp,open,high,low,close
datetime,,,,,
2023-11-09 23:25:00,1699572300000,36534.5,36619.1,36532.5,36610.0
2023-11-09 23:30:00,1699572600000,36610.0,36695.0,36597.7,36652.4
2023-11-09 23:35:00,1699572900000,36652.3,36728.0,36652.3,36697.0
2023-11-09 23:40:00,1699573200000,36697.1,36778.0,36642.4,36762.8
2023-11-09 23:45:00,1699573500000,36762.9,36768.0,36690.2,36750.9
...,...,...,...,...,...
2023-11-10 15:40:00,1699630800000,37039.7,37048.8,36950.0,36962.6
2023-11-10 15:45:00,1699631100000,36962.6,37080.0,36945.0,37077.1
2023-11-10 15:50:00,1699631400000,37077.1,37128.6,37030.8,37033.4


In [45]:
binance_test = BinanceUSDM(
    api_key=BinanceUSDMTestKeys.api_key,
    secret_key=BinanceUSDMTestKeys.secret_key,
    use_test_net=True,
)
binance_test.create_order(
    symbol="BTCUSDT",
    side="BUY",
    type="MARKET",
    quantity=0.005,
    timeInForce=None,
)